In [1]:
from dask.diagnostics import Profiler 

In [2]:
import dask
import h5py
import os
import pandas as pd
import sys
import dask.threaded
from dask import compute, delayed 
import dask.dataframe as ddf

In [3]:
data_dir = "../data/"

There contains some 

In [5]:
pd_h5_files = [data_dir + file for file in os.listdir(data_dir) 
               if 'h5' in file]
pd_csv_files = [data_dir + file for file in os.listdir(data_dir)
                if "csv" in file and '2001' not in file and 
                '2003' not in file]
csv_files = [data_dir + file for file in os.listdir(data_dir)
                if "csv" in file and 'gzip' not in file]
csv_dfs = ddf.read_csv('../data/19*.csv')

# read everything serially as a baseline

In [9]:
pd_h5_files[:5]

['../data/1987_gzip.h5',
 '../data/1988_gzip.h5',
 '../data/1989_gzip.h5',
 '../data/1990_gzip.h5',
 '../data/1991_gzip.h5']

In [ ]:
%time h5_df = [pd.read_hdf(file, 'csv') for file in pd_h5_files]

In [7]:
%time h5_df = [pd.read_hdf(file, 'csv') for file in pd_h5_files[:5]]

CPU times: user 1min 6s, sys: 10.9 s, total: 1min 17s
Wall time: 1min 31s


In [10]:
overall_df = pd.concat(h5_df)

In [13]:
overall_df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [14]:
overall_df.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,TailNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,2.190294e+07,2.190294e+07,2.190294e+07,2.190294e+07,2.166296e+07,2.190294e+07,2.160134e+07,2.190294e+07,2.190294e+07,0.0,...,0.0,0.0,2.190294e+07,0.0,2.190294e+07,0.0,0.0,0.0,0.0,0.0
mean,1.989347e+03,6.786259e+00,1.571979e+01,3.943505e+00,1.359918e+03,1.352823e+03,1.494677e+03,1.494819e+03,7.423620e+02,NaN,...,NaN,NaN,1.095634e-02,NaN,2.813732e-03,NaN,NaN,NaN,NaN,NaN
std,1.234717e+00,3.508681e+00,8.789153e+00,1.990197e+00,4.773678e+02,4.712698e+02,4.954547e+02,4.847782e+02,5.459095e+02,NaN,...,NaN,NaN,1.040975e-01,NaN,5.296994e-02,NaN,NaN,NaN,NaN,NaN
min,1.987000e+03,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,...,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
25%,1.988000e+03,4.000000e+00,8.000000e+00,2.000000e+00,9.390000e+02,9.350000e+02,1.117000e+03,1.116000e+03,3.210000e+02,NaN,...,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
50%,1.989000e+03,7.000000e+00,1.600000e+01,4.000000e+00,1.339000e+03,1.333000e+03,1.522000e+03,1.520000e+03,6.240000e+02,NaN,...,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
75%,1.990000e+03,1.000000e+01,2.300000e+01,6.000000e+00,1.747000e+03,1.740000e+03,1.918000e+03,1.915000e+03,1.056000e+03,NaN,...,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN
max,1.991000e+03,1.200000e+01,3.100000e+01,7.000000e+00,2.400000e+03,2.400000e+03,2.400000e+03,2.400000e+03,9.851000e+03,NaN,...,NaN,NaN,1.000000e+00,NaN,1.000000e+00,NaN,NaN,NaN,NaN,NaN


# try working on embarassingly parallel tasks

summary:

the entire kernel dies 

* no fault tolerance for Dask multi-threading. If one of the many multi-threaded tasks fails, every task fails.
* no fault tolerance for Dask dataframes, 
* 

In [5]:
# csv_dfs[:2].ArrDelay.compute()

In [15]:
def read_csv_with_cleaning(filepath):
    """handles IO error due to bad data type within files 
    """
    try:
        return pd.read_csv(file)
    except "UnicodeDecodeError":
        # try just returning nothing
        return None

In [8]:
len(pd_h5_files)

22

In [38]:
len(df_delayed_content)

21

In [39]:
df_delayed_content

[Delayed('read_csv-3bf1731a-f1f0-4032-a8f8-56c9b5171666'),
 Delayed('read_csv-7f3ef660-0636-4ddc-8c12-587b4bf8ab97'),
 Delayed('read_csv-6a25795a-11fc-452d-8806-a181d184c07d'),
 Delayed('read_csv-7c5509fe-0182-4b80-b432-bddf105c3848'),
 Delayed('read_csv-cbdc6015-5028-45ee-b997-46b0d1160eff'),
 Delayed('read_csv-717ab110-b00e-4ceb-9752-bb154cd089de'),
 Delayed('read_csv-fd69537c-859d-4486-b9d6-065f9d9cdbc3'),
 Delayed('read_csv-f6259c36-ea5b-4f9f-bf52-ff79aaa827a1'),
 Delayed('read_csv-fdcf26f5-0ac6-4b8c-81fe-aec395a65d33'),
 Delayed('read_csv-5aae3848-b596-4703-9a53-8f5cd18a9dd1'),
 Delayed('read_csv-cc24566c-88f4-4c08-8964-8a8a32cee08b'),
 Delayed('read_csv-f737ca91-a54a-4466-86fd-8e4f57982908'),
 Delayed('read_csv-84440da3-a79a-4686-8226-b7219fd28294'),
 Delayed('read_csv-72591628-a7cb-4387-ac71-20685404a093'),
 Delayed('read_csv-971c6fd8-641e-4e1e-beaf-738daabe1e6d'),
 Delayed('read_csv-0cce2406-c3a4-47eb-91e3-41f7e5b1babd'),
 Delayed('read_csv-2bab4f06-f82e-4cdc-a871-846a89a7c004'

one of the many csv files is corrupted

In [33]:
%pdb

Automatic pdb calling has been turned ON


In [7]:
%time df_content = compute(*df_delayed_content[:3], get=dask.threaded.get)

CPU times: user 27.9 s, sys: 13.9 s, total: 41.8 s
Wall time: 12.6 s


In [17]:
%time df_content = compute(*df_delayed_content, get=dask.multiprocessing.get)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 4: invalid continuation byte

Traceback
---------
  File "/home/karenyin/miniconda3/py35_envs/idp35_201703/lib/python3.5/site-packages/dask/local.py", line 290, in execute_task
    result = _execute_task(task, data)
  File "/home/karenyin/miniconda3/py35_envs/idp35_201703/lib/python3.5/site-packages/dask/local.py", line 271, in _execute_task
    return func(*args2)
  File "/home/karenyin/miniconda3/py35_envs/idp35_201703/lib/python3.5/site-packages/pandas/io/parsers.py", line 655, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "/home/karenyin/miniconda3/py35_envs/idp35_201703/lib/python3.5/site-packages/pandas/io/parsers.py", line 411, in _read
    data = parser.read(nrows)
  File "/home/karenyin/miniconda3/py35_envs/idp35_201703/lib/python3.5/site-packages/pandas/io/parsers.py", line 982, in read
    ret = self._engine.read(nrows)
  File "/home/karenyin/miniconda3/py35_envs/idp35_201703/lib/python3.5/s

In [14]:
%time df_content = compute(*df_delayed_content, get=dask.multiprocessing.get)

CPU times: user 6.11 s, sys: 12.3 s, total: 18.4 s
Wall time: 2min 7s


In [15]:
df_content

(         Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  ArrTime  \
 0        1987     10          14          3    741.0         730    912.0   
 1        1987     10          15          4    729.0         730    903.0   
 2        1987     10          17          6    741.0         730    918.0   
 3        1987     10          18          7    729.0         730    847.0   
 4        1987     10          19          1    749.0         730    922.0   
 5        1987     10          21          3    728.0         730    848.0   
 6        1987     10          22          4    728.0         730    852.0   
 7        1987     10          23          5    731.0         730    902.0   
 8        1987     10          24          6    744.0         730    908.0   
 9        1987     10          25          7    729.0         730    851.0   
 10       1987     10          26          1    735.0         730    904.0   
 11       1987     10          28          3    741.0         72

In [11]:
pd.concat(df_content)

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1987,10,14,3,741.0,730,912.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1987,10,15,4,729.0,730,903.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1987,10,17,6,741.0,730,918.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1987,10,18,7,729.0,730,847.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1987,10,19,1,749.0,730,922.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
5,1987,10,21,3,728.0,730,848.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
6,1987,10,22,4,728.0,730,852.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
7,1987,10,23,5,731.0,730,902.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
8,1987,10,24,6,744.0,730,908.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
9,1987,10,25,7,729.0,730,851.0,849,PS,1451,...,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [13]:
df = pd.read_csv('../data/1987.csv.bz2')

In [16]:
delayReasons = ['CarrierDelay',
               'WeatherDelay', 
               'NASDelay', 
               'SecurityDelay', 
               'LateAircraftDelay']

In [ ]:
pd.read_csv()

In [17]:
df[delayReasons]

,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN


In [ ]:
kk